# Circuit learning module: Pennylane

In [1]:
import json
import os
import glob
import warnings
from pathlib import Path
import numpy as np
import pickle
import pennylane as qml
from sympy import default_sort_key
import torch
from discopy.quantum.pennylane import to_pennylane, PennyLaneCircuit
from inspect import signature

this_folder = os.path.abspath(os.getcwd())

ModuleNotFoundError: No module named 'pennylane'

## Read circuit data

We read the circuits from the pickled files.

In [ ]:
training_circuits_paths = glob.glob(this_folder + "//simplified-JOB-diagrams//circuits//binary_classification//training//[0-9]*.p")
test_circuits_paths = glob.glob(this_folder + "//simplified-JOB-diagrams//circuits//binary_classification//test//[0-9]*.p")

def read_diagrams(circuit_paths):
    circuits = {}
    for serialized_diagram in circuit_paths:
        base_name = Path(serialized_diagram).stem
        f = open(serialized_diagram, "rb")
        diagram = pickle.load(f)
        circuits[base_name] = diagram
    return circuits


training_circuits = read_diagrams(training_circuits_paths)
test_circuits = read_diagrams(test_circuits_paths)

print("Number of training circuits: ", len(training_circuits))
print("Number of test circuits: ", len(test_circuits))

## Read training and test data

In [ ]:
training_data, test_data = None, None
with open(this_folder + "//data//training_data.json", "r") as inputfile:
    training_data = json.load(inputfile)['training_data']
with open(this_folder + "//data//test_data.json", "r") as inputfile:
    test_data = json.load(inputfile)['test_data']
    

def time_to_states(data):
    labeled_data = {}
    for elem in data:
        if elem["time"] < 2001:
            labeled_data[elem["name"]] = [1,0] # corresponds to |0>
        else:
            labeled_data[elem["name"]] = [0,1] # corresponds to |1>
    return labeled_data


training_data_labels = time_to_states(training_data)
test_data_labels = time_to_states(test_data)

## Construct circuits in Pennylane

Next we translate DisCoPy circuits in Pennylane and rewrite the Pennylane circuits. By default the encoded measurement function in the Pennylane circuit function is `state()` or `probs()` which we change to `sampe()`. We use `sample()` because we need to do post-selection which is not implemented in Pennylane. Technically we also have the problem that Pennylane does not natively support training over circuits which have varying signature (parameters) and varying number of wires. Thus we rewrite the circuits so that at each step, all the circuits accept all the parameters as input but they will utilize only a subset of them. Because the parameters need to be inputed as a vector, for each circuit we calculate a signature tuple, which encodes the information of the needed parameters in the circuit.

The code assumes that the DisCoPy circuit contains only such gates that accept a single parameter or no parameters.

$$ \texttt{parameter symbol in circuit} \to \texttt{index among all parameters in model} \to \texttt{value from input for circuit} $$

In [ ]:
tot_qubits = 20
dev = qml.device("default.qubit", wires=tot_qubits, shots=4)

qml_circuits_training = []
qml_circuits_test = []
symbols = set([elem for c in training_circuits for elem in training_circuits[c].free_symbols])
symbols = list(sorted(symbols, key=default_sort_key))

for c in training_circuits:
    circuit_diagram = training_circuits[c]
    pennylane_circuit = to_pennylane(circuit_diagram)
    params = pennylane_circuit.params
    pennylane_wires = pennylane_circuit.wires
    ops = pennylane_circuit.ops
    param_symbols = [[sym[0].as_ordered_factors()[2]] if len(sym) > 0 else [] for sym in params]
    symbol_to_index = {}

    for sym in param_symbols:
        if len(sym) > 0:
            symbol_to_index[sym[0]] = symbols.index(sym[0])

    @qml.qnode(dev, interface="torch")
    def qml_circuit(circ_params):
        for op, param, wires in zip(ops, param_symbols, pennylane_wires):
            if len(param) > 0:
                param = param[0]
                op(circ_params[symbol_to_index[param]], wires = wires)
            else:
                op(wires = wires)
        return qml.sample()
    
    qml_circuits_training.append(qml_circuit)

training_qnodes = qml.QNodeCollection(qml_circuits_training)

In [ ]:
SEED = 0
rng = np.random.default_rng(SEED)
x0 = np.array(rng.random(len(symbols)))
#np.random.seed(SEED)
#x0 = [torch.FloatTensor([x]) for x in x0]
print(len(x0))

In [ ]:
result = training_qnodes(x0)

In [ ]:
print(len(result[0][1]))

## Post-selection

In [ ]:
def post_select(shot, post_selection):
    diff = len(shot) - len(post_selection)
    for i in range(len(post_selection)):
        if shot[i + diff] != post_selection[i]:
            return []
    else:
        return shot

In [ ]:
#for res in result:
#    for shot in res:
#        print(post_select(shot, [0]*(len(shot) - 1)))

## Cost function

In [ ]:
def cost_spsa(params):
    sample_set = training_qnodes(params)
    selected = []
    for shot in sample:
        selected += post_select(shot, post_selection)
    print(selected)
    return None #qnode_spsa(params.reshape(num_layers, num_wires, 3))